In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from time import sleep
import json
import logging
import datetime
from selenium.webdriver.chrome.options import Options

import subprocess
import gzip
import os
import json
from time import sleep

from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import json
import datetime
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
from time import sleep
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import random


In [2]:
def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data



import re

def get_competition(text):
    # 假设你已经用之前的方法获得了这个字符串,字符串样例示例：
    #text = "（排名129/364）"
    # 使用正则表达式匹配文本中的数字
    matches = re.findall(r'\d+', text)

    # 假设文本格式固定，排名和总人数都是数字，并且总人数在第二个位置
    if matches:
        total_number = matches[1]  # 获取总人数
        
    return total_number



def transform_salary_description(salary_desc):
    # 检查是否包含"及以下"，以及是千还是万
    if '千及以下' in salary_desc:
        value, _ = salary_desc.split('千及以下')
        min_value = round(float(value) * 0.6, 5)  # 计算下限并四舍五入到5位小数
        new_salary_desc = f"{min_value}-{value}千"
    elif '万及以下' in salary_desc:
        value, _ = salary_desc.split('万及以下')
        min_value = round(float(value) * 0.6, 5)  # 计算下限并四舍五入到5位小数
        new_salary_desc = f"{min_value}-{value}万"
    else:
        # 如果不符合上述任一条件，则不需要转换
        return salary_desc
    
    return new_salary_desc




def parse_and_average_salary(salary):
    # 将薪资字符串中的“千”和“万”统一转换为浮点数（以“万”为单位）
    salary = salary.replace('千', '*0.1').replace('万', '')
    # 分割薪资范围
    salary_range = salary.split('-')
    # 计算薪资范围的平均值
    if len(salary_range) == 2:
        min_salary, max_salary = salary_range
        average_salary = ((eval(min_salary) + eval(max_salary)) * 10000) / 2
    else:
        average_salary = eval(salary_range[0]) * 10000
    return round(average_salary , 2)  # 返回平均薪资，单位为元


def parse_and_average_salary_true(salary):
    # 将薪资字符串中的“千”和“万”统一转换为浮点数（以“万”为单位）
    if "万" in salary:
        salary = salary.replace('千', '*0.1').replace('万', '')
        # 分割薪资范围
        salary_range = salary.split('-')
        # 计算薪资范围的平均值
        if len(salary_range) == 2:
            min_salary, max_salary = salary_range
            average_salary = ((eval(min_salary) + eval(max_salary)) * 10000) / 2
        else:
            average_salary = eval(salary_range[0]) * 10000
        return round(average_salary , 2)  # 返回平均薪资，单位为元
    #仅含"千"
    else:
        salary = salary.replace('千', '')
        salary_range = salary.split('-')
        if len(salary_range) == 2:
            min_salary, max_salary = salary_range
            average_salary = ((eval(min_salary) + eval(max_salary)) * 1000) / 2
        else:
            average_salary = eval(salary_range[0]) * 1000
        return round(average_salary , 2)  # 返回平均薪资，单位为元



def calculate_monthly_salary_true(salary_string):
    # 假设transform_salary_description已经将薪资描述转换为了易于解析的格式
    salary_string=transform_salary_description(salary_string)
    if '年' in salary_string:
        salary_string = salary_string.replace('/年', '')
        average_annual_salary = parse_and_average_salary_true(salary_string)
        return round(average_annual_salary / 12, 2)  # 转换为月薪
    elif '薪' in salary_string:
        salary_range, months = salary_string.split('·')
        average_salary = parse_and_average_salary_true(salary_range)
        months = float(months.replace('薪', ''))
        return round(average_salary * months / 12, 2)  # 计算年薪再转换为月薪
    elif '-' in salary_string:
        return parse_and_average_salary_true(salary_string)  # 直接返回月薪
    else:
        # 不属于上述情况，可能需要进一步处理
        return None



def calculate_monthly_salary(salary_string):
    # 假设transform_salary_description已经将薪资描述转换为了易于解析的格式
    salary_string=transform_salary_description(salary_string)
    if '年' in salary_string:
        salary_string = salary_string.replace('/年', '')
        average_annual_salary = parse_and_average_salary(salary_string)
        return round(average_annual_salary / 12, 2)  # 转换为月薪
    elif '薪' in salary_string:
        salary_range, months = salary_string.split('·')
        average_salary = parse_and_average_salary(salary_range)
        months = float(months.replace('薪', ''))
        return round(average_salary * months / 12, 2)  # 计算年薪再转换为月薪
    elif '-' in salary_string:
        return parse_and_average_salary(salary_string)  # 直接返回月薪
    else:
        # 不属于上述情况，可能需要进一步处理
        return None


In [3]:

options = ChromeOptions()
""" #使用代理服务器，便于用Mitmproxy监听
options.add_argument("--proxy-server=http://localhost:8080") """
# 忽略SSL错误
options.add_argument("--ignore-ssl-errors=yes")
options.add_argument("--ignore-certificate-errors")
#禁用Blink引擎的某些特性，未指定
options.add_argument("--disable-blink-features")
#隐藏Selenium自动化控制的痕迹
options.add_argument("--disable-blink-features=AutomationControlled")

############
#无界面模式
#options.add_argument("--headless")
############

chromedriver_path = r"C:\chromedriver-win64\chromedriver.exe"


""" try:
    times = int(input("输入需要爬取的数量，没有输入则爬取所有岗位"))
except ValueError:
    times = float('inf')  # 设置为无限大，表示爬取所有职位
 """
times = float('inf')

# 创建Service对象，并传入ChromeDriver的路径
service = Service(executable_path=chromedriver_path)
# 使用Service对象初始化Chrome WebDriver
driver = webdriver.Chrome(service=service,options=options)

wait = WebDriverWait(driver, 60)


In [3]:
# 找到页面上所有特定的元素
elements = driver.find_elements(By.CSS_SELECTOR, ".e")

job_listings = []

# 遍历每个元素，提取并打印/保存你感兴趣的信息
for element in elements:
    try:
        job_title = element.find_element(By.CSS_SELECTOR, "a.zhn.gt").text
        salary = element.find_element(By.CSS_SELECTOR, "span.xz").text
        company = element.find_element(By.CSS_SELECTOR, "a.gs").text
        location = element.find_element(By.CSS_SELECTOR, "span.dq").text

        # 提取申请时间和简历状态
        application_time = element.find_element(By.CSS_SELECTOR, ".li.l2 .rq span").text
        resume_status = element.find_element(By.CSS_SELECTOR, ".li.l4 .view span").text
        
        # #print(f"Job Title: {job_title}, Salary: {salary}, Company: {company}, Location: {location}")
        # print(f"Application Time: {application_time}, Resume Status: {resume_status}")
        
        # #application_time = elements[i].find_element(By.CSS_SELECTOR, "div.li.l2 .rq span").text
        # #resume_status = elements[i].find_element(By.CSS_SELECTOR, "div.h_body").text.split('\n')[0]  # 假设状态为第一行文本
        
        # print(f"Job Title: {job_title}, Salary: {salary}, Company: {company}, Location: {location}")    
                # 创建包含所有信息的字典
                
        job_info = {
            "Job Title": job_title,
            "Salary": salary,
            "Company": company,
            "Location": location,
            "Application Time": application_time,
            "Resume Status": resume_status
        }
        
        # 将字典添加到列表中
        job_listings.append(job_info)

    except:
        continue


# # 将列表中的字典写入JSON文件
# with open('job_listings.json', 'w', encoding='utf-8') as f:
#     json.dump(job_listings, f, ensure_ascii=False, indent=4)

In [47]:
    # 查找并点击下一页按钮
next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")
next_page_button.click()
    

In [11]:
# List to store job listings
job_listings = []


while True:

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".e")))

    # 找到页面上所有特定的元素
    elements = driver.find_elements(By.CSS_SELECTOR, ".e")
    
        # 遍历每个元素，提取并保存感兴趣的信息
    for element in elements:
        try:
            job_title = element.find_element(By.CSS_SELECTOR, "a.zhn.gt").text
            salary = element.find_element(By.CSS_SELECTOR, "span.xz").text
            company = element.find_element(By.CSS_SELECTOR, "a.gs").text
            location = element.find_element(By.CSS_SELECTOR, "span.dq").text
            application_time = element.find_element(By.CSS_SELECTOR, ".li.l2 .rq span").text
            resume_status = element.find_element(By.CSS_SELECTOR, ".li.l4 .view span").text
            
            # 创建包含所有信息的字典
            job_info = {
                "Job Title": job_title,
                "Salary": salary,
                "Company": company,
                "Location": location,
                "Application Time": application_time,
                "Resume Status": resume_status
            }
            
            # 将字典添加到列表中
            job_listings.append(job_info)
        except:
            continue



    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".e")))

    # 查找并点击下一页按钮
    next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")

    next_page_button.click()

    # 等待新页面加载完成
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".e")))
    sleep(5)
    
    # 如果你想自动停止，可以在这里添加一个条件来break循环


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=123.0.6312.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60F52AD32+56930]
	(No symbol) [0x00007FF60F49F632]
	(No symbol) [0x00007FF60F3542E5]
	(No symbol) [0x00007FF60F363EB1]
	(No symbol) [0x00007FF60F359FB4]
	(No symbol) [0x00007FF60F35A139]
	(No symbol) [0x00007FF60F35827A]
	(No symbol) [0x00007FF60F35B9FD]
	(No symbol) [0x00007FF60F3D8E57]
	(No symbol) [0x00007FF60F3BBC9A]
	(No symbol) [0x00007FF60F3D81E2]
	(No symbol) [0x00007FF60F3BBA43]
	(No symbol) [0x00007FF60F38D438]
	(No symbol) [0x00007FF60F38E4D1]
	GetHandleVerifier [0x00007FF60F8A6ABD+3709933]
	GetHandleVerifier [0x00007FF60F8FFFFD+4075821]
	GetHandleVerifier [0x00007FF60F8F818F+4043455]
	GetHandleVerifier [0x00007FF60F5C9766+706710]
	(No symbol) [0x00007FF60

In [20]:
# List to store job listings
job_listings = []

# 获取总页数
total_pages_element = driver.find_element(By.XPATH, "//div[@class='dw_page']//span[contains(@class, 'td')]")
total_pages = int(total_pages_element.text.split('共')[-1].split('页')[0])

# 当前页数
current_page = 1

# 循环，直到你决定停止手动翻页
while current_page <= total_pages:
    # 找到页面上所有特定的元素
    elements = driver.find_elements(By.CSS_SELECTOR, ".e")
    
        # 遍历每个元素，提取并保存感兴趣的信息
    for element in elements:
        try:
            job_title = element.find_element(By.CSS_SELECTOR, "a.zhn.gt").text
            salary = element.find_element(By.CSS_SELECTOR, "span.xz").text
            company = element.find_element(By.CSS_SELECTOR, "a.gs").text
            location = element.find_element(By.CSS_SELECTOR, "span.dq").text
            application_time = element.find_element(By.CSS_SELECTOR, ".li.l2 .rq span").text
            resume_status = element.find_element(By.CSS_SELECTOR, ".li.l4 .view span").text
            
            # 创建包含所有信息的字典
            job_info = {
                "Job Title": job_title,
                "Salary": salary,
                "Company": company,
                "Location": location,
                "Application Time": application_time,
                "Resume Status": resume_status
            }
            
            # 将字典添加到列表中
            job_listings.append(job_info)
        except:
            continue
    

    # 查找下一页按钮
    next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")
    
    # 点击下一页按钮
    next_page_button.click()
    
    # 等待新页面加载完成
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".e")))
    
    # 更新当前页数
    current_page += 1

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='dw_page']//a[text()='下一页']"}
  (Session info: chrome=123.0.6312.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60F52AD32+56930]
	(No symbol) [0x00007FF60F49F632]
	(No symbol) [0x00007FF60F3542E5]
	(No symbol) [0x00007FF60F3998ED]
	(No symbol) [0x00007FF60F399A2C]
	(No symbol) [0x00007FF60F3DA967]
	(No symbol) [0x00007FF60F3BBCDF]
	(No symbol) [0x00007FF60F3D81E2]
	(No symbol) [0x00007FF60F3BBA43]
	(No symbol) [0x00007FF60F38D438]
	(No symbol) [0x00007FF60F38E4D1]
	GetHandleVerifier [0x00007FF60F8A6ABD+3709933]
	GetHandleVerifier [0x00007FF60F8FFFFD+4075821]
	GetHandleVerifier [0x00007FF60F8F818F+4043455]
	GetHandleVerifier [0x00007FF60F5C9766+706710]
	(No symbol) [0x00007FF60F4AB90F]
	(No symbol) [0x00007FF60F4A6AF4]
	(No sym

In [32]:
def get_information_from_elements(driver,job_listings):
    elements = driver.find_elements(By.CSS_SELECTOR, ".e")

        # 遍历每个元素，提取并保存感兴趣的信息
    for element in elements:
        try:
            job_title = element.find_element(By.CSS_SELECTOR, "a.zhn.gt").text
            salary = element.find_element(By.CSS_SELECTOR, "span.xz").text
            company = element.find_element(By.CSS_SELECTOR, "a.gs").text
            location = element.find_element(By.CSS_SELECTOR, "span.dq").text
            application_time = element.find_element(By.CSS_SELECTOR, ".li.l2 .rq span").text
            resume_status = element.find_element(By.CSS_SELECTOR, ".li.l4 .view span").text
            
            # 创建包含所有信息的字典
            job_info = {
                "Job Title": job_title,
                "Salary": salary,
                "Company": company,
                "Location": location,
                "Application Time": application_time,
                "Resume Status": resume_status
            }
            
            # 将字典添加到列表中
            job_listings.append(job_info)
        except:
            continue
    return job_listings


# List to store job listings
job_listings = []

# 获取总页数
total_pages_element = driver.find_element(By.XPATH, "//div[@class='dw_page']//span[contains(@class, 'td')]")
total_pages = int(total_pages_element.text.split('共')[-1].split('页')[0])

# 当前页数
current_page = 1

# 循环,遍历所有页面
while current_page <= total_pages:
    if current_page< total_pages:
        # 找到页面上所有特定的元素
        job_listings=get_information_from_elements(driver,job_listings)

        sleep(2)
        try:
            # 查找下一页按钮
            next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")
        except NoSuchElementException:
            sleep(3)
            next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")
            
        # 如果下一页按钮是不可点击的 span 元素，则说明当前页面已经是最后一页，退出循环
        if 'bk' in next_page_button.get_attribute('class'):
            break
        
        # 使用 JavaScript 来模拟点击下一页按钮
        driver.execute_script("arguments[0].click();", next_page_button)
        
        # 等待新页面加载完成
        sleep(5)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".e")))
        
        # 更新当前页数
        current_page += 1
    else:
        # 找到页面上所有特定的元素
        job_listings=get_information_from_elements(driver,job_listings)
        current_page += 1




In [21]:

def get_information_from_elements(driver,job_listings):
    elements = driver.find_elements(By.CSS_SELECTOR, ".txt")
    # 遍历找到的元素
    for element in elements:
        try:
            # 获取公司名称
            company_name = element.find_element(By.CSS_SELECTOR, ".li.l1 a").text
            
            # 获取HR兴趣度的宽度
            interest_width_style = element.find_element(By.CSS_SELECTOR, ".like span").get_attribute('style')
            interest_width = interest_width_style.split('width:')[1].replace('px;', '').strip()
            
            # 获取查看次数
            views_count = element.find_element(By.CSS_SELECTOR, ".li.l3 strong.c_orange").text
            
            # 获取最后一次查看的时间
            last_view_time = element.find_element(By.CSS_SELECTOR, ".li.l3 .f12 span.c_orange").text
            
            job_info={
                'company_name': company_name,
                'interest_width': interest_width,
                'views_count': views_count,
                'last_view_time': last_view_time
            }
            # 把提取的数据添加到列表中
            job_listings.append(job_info)

        except NoSuchElementException:
            # 如果没有找到某个元素，输出错误信息并跳过当前元素
            # print("Could not find one of the elements. Skipping.")
            continue
    return job_listings


job_listings = []


# 获取总页数
total_pages_element = driver.find_element(By.XPATH, "//div[@class='dw_page']//span[contains(@class, 'td')]")
total_pages = int(total_pages_element.text.split('共')[-1].split('页')[0])

# 当前页数
current_page = 1

# 循环,遍历所有页面
while current_page <= total_pages:
    if current_page< total_pages:
        # 找到页面上所有特定的元素
        job_listings=get_information_from_elements(driver,job_listings)

        sleep(2)
        try:
            # 查找下一页按钮
            next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")
        except NoSuchElementException:
            sleep(3)
            next_page_button = driver.find_element(By.XPATH, "//div[@class='dw_page']//a[text()='下一页']")
            
        # 如果下一页按钮是不可点击的 span 元素，则说明当前页面已经是最后一页，退出循环
        if 'bk' in next_page_button.get_attribute('class'):
            break
        
        # 使用 JavaScript 来模拟点击下一页按钮
        driver.execute_script("arguments[0].click();", next_page_button)
        
        # 等待新页面加载完成
        sleep(5)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".e")))
        
        # 更新当前页数
        current_page += 1
    else:
        # 找到页面上所有特定的元素
        job_listings=get_information_from_elements(driver,job_listings)
        current_page += 1


KeyboardInterrupt: 

In [18]:

job_listings = []


Could not find one of the elements. Skipping.
Could not find one of the elements. Skipping.
Could not find one of the elements. Skipping.
Could not find one of the elements. Skipping.
{'company_name': '上海臻寰实业有限公司', 'interest_width': '8', 'views_count': '1', 'last_view_time': '2024-04-11 16:00'}
{'company_name': '广州南沙开发建设集团有限公司', 'interest_width': '8', 'views_count': '1', 'last_view_time': '2024-04-11 16:00'}
{'company_name': '上海恒信电脑软件有限公司', 'interest_width': '8', 'views_count': '1', 'last_view_time': '2024-04-11 16:00'}
{'company_name': '无穷食品制造（广东）有限公司', 'interest_width': '8', 'views_count': '1', 'last_view_time': '2024-04-11 16:00'}
{'company_name': '深圳市雷能混合集成电路有限公司', 'interest_width': '8', 'views_count': '1', 'last_view_time': '2024-04-11 16:00'}
{'company_name': '上海起迪计算机科技发展有限公司', 'interest_width': '8', 'views_count': '1', 'last_view_time': '2024-04-11 16:00'}
{'company_name': '上海微凌信息科技有限公司', 'interest_width': '8', 'views_count': '1', 'last_view_time': '2024-04-11 16:00'}
{'company_

In [20]:
job_listings

[{'company_name': '上海臻寰实业有限公司',
  'interest_width': '8',
  'views_count': '1',
  'last_view_time': '2024-04-11 16:00'},
 {'company_name': '广州南沙开发建设集团有限公司',
  'interest_width': '8',
  'views_count': '1',
  'last_view_time': '2024-04-11 16:00'},
 {'company_name': '上海恒信电脑软件有限公司',
  'interest_width': '8',
  'views_count': '1',
  'last_view_time': '2024-04-11 16:00'},
 {'company_name': '无穷食品制造（广东）有限公司',
  'interest_width': '8',
  'views_count': '1',
  'last_view_time': '2024-04-11 16:00'},
 {'company_name': '深圳市雷能混合集成电路有限公司',
  'interest_width': '8',
  'views_count': '1',
  'last_view_time': '2024-04-11 16:00'},
 {'company_name': '上海起迪计算机科技发展有限公司',
  'interest_width': '8',
  'views_count': '1',
  'last_view_time': '2024-04-11 16:00'},
 {'company_name': '上海微凌信息科技有限公司',
  'interest_width': '8',
  'views_count': '1',
  'last_view_time': '2024-04-11 16:00'},
 {'company_name': '朗芯微电子（深圳）有限公司',
  'interest_width': '8',
  'views_count': '1',
  'last_view_time': '2024-04-11 16:00'},
 {'company_name

In [13]:
elements

[<selenium.webdriver.remote.webelement.WebElement (session="bc39a1f80551de7b135fadc48e2200ce", element="f.287F1BE5C7906E4AD21E8A18BF7D49E7.d.C16929F64C1C4CEBE8D35F7658D6E7F1.e.786")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bc39a1f80551de7b135fadc48e2200ce", element="f.287F1BE5C7906E4AD21E8A18BF7D49E7.d.C16929F64C1C4CEBE8D35F7658D6E7F1.e.789")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bc39a1f80551de7b135fadc48e2200ce", element="f.287F1BE5C7906E4AD21E8A18BF7D49E7.d.C16929F64C1C4CEBE8D35F7658D6E7F1.e.739")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bc39a1f80551de7b135fadc48e2200ce", element="f.287F1BE5C7906E4AD21E8A18BF7D49E7.d.C16929F64C1C4CEBE8D35F7658D6E7F1.e.743")>,
 <selenium.webdriver.remote.webelement.WebElement (session="bc39a1f80551de7b135fadc48e2200ce", element="f.287F1BE5C7906E4AD21E8A18BF7D49E7.d.C16929F64C1C4CEBE8D35F7658D6E7F1.e.747")>]

In [16]:

    # 遍历每个元素，提取并保存感兴趣的信息
for element in elements:
    try:
        # job_title = element.find_element(By.CSS_SELECTOR, "a.zhn.gt").text
        # salary = element.find_element(By.CSS_SELECTOR, "span.xz").text
        company = element.find_element(By.CSS_SELECTOR, "li.l1").title
        # location = element.find_element(By.CSS_SELECTOR, "span.dq").text
        # application_time = element.find_element(By.CSS_SELECTOR, ".li.l2 .rq span").text
        # resume_status = element.find_element(By.CSS_SELECTOR, ".li.l4 .view span").text
        
        # # 创建包含所有信息的字典
        # job_info = {
        #     "Job Title": job_title,
        #     "Salary": salary,
        #     "Company": company,
        #     "Location": location,
        #     "Application Time": application_time,
        #     "Resume Status": resume_status
        # }
        
        # # 将字典添加到列表中
        # job_listings.append(job_info)
    except:
        continue

IndentationError: expected an indented block after 'for' statement on line 2 (105072326.py, line 3)

In [28]:
# 提取简历状态
resume_status_element = element.find_element(By.CSS_SELECTOR, ".li.l4 .view span")
resume_status = resume_status_element.text


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=123.0.6312.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60F52AD32+56930]
	(No symbol) [0x00007FF60F49F632]
	(No symbol) [0x00007FF60F3542E5]
	(No symbol) [0x00007FF60F363EB1]
	(No symbol) [0x00007FF60F359FB4]
	(No symbol) [0x00007FF60F35827A]
	(No symbol) [0x00007FF60F35B6EB]
	(No symbol) [0x00007FF60F35B7B0]
	(No symbol) [0x00007FF60F39955C]
	(No symbol) [0x00007FF60F399A2C]
	(No symbol) [0x00007FF60F38F13C]
	(No symbol) [0x00007FF60F3BBCDF]
	(No symbol) [0x00007FF60F38F09A]
	(No symbol) [0x00007FF60F3BBEB0]
	(No symbol) [0x00007FF60F3D81E2]
	(No symbol) [0x00007FF60F3BBA43]
	(No symbol) [0x00007FF60F38D438]
	(No symbol) [0x00007FF60F38E4D1]
	GetHandleVerifier [0x00007FF60F8A6ABD+3709933]
	GetHandleVerifier [0x00007FF60F

In [26]:

# 将列表中的字典写入JSON文件
with open('job_listings.json', 'w', encoding='utf-8') as f:
    json.dump(job_listings, f, ensure_ascii=False, indent=4)

In [41]:
job_listings

[{'Job Title': '光伏技术工程师(J10083)',
  'Salary': '7-8.5千·18薪',
  'Company': '深圳创维光伏科技有限公司',
  'Location': '深圳-光明区',
  'Application Time': '2024-04-08',
  'Resume Status': '已到达'},
 {'Job Title': '数据分析专员',
  'Salary': '8-9千',
  'Company': '上海共添市场营销服务股份有限公司',
  'Location': '上海',
  'Application Time': '2024-04-08',
  'Resume Status': '已到达'},
 {'Job Title': '项目经理',
  'Salary': '8-9千',
  'Company': '深圳高速公路石油有限公司',
  'Location': '深圳-福田区',
  'Application Time': '2024-04-08',
  'Resume Status': '已到达'},
 {'Job Title': '项目助理',
  'Salary': '7-8千·15薪',
  'Company': '森欧汽车内饰材料（江苏）有限公司',
  'Location': '广州-番禺区',
  'Application Time': '2024-04-08',
  'Resume Status': '已查阅'},
 {'Job Title': 'c证货运司机',
  'Salary': '1.3-1.8万',
  'Company': '上海瑜源货物运输代理有限公司',
  'Location': '上海',
  'Application Time': '2024-04-08',
  'Resume Status': '已到达'},
 {'Job Title': 'IT基础架构工程师',
  'Salary': '1-1.5万',
  'Company': '上海电子工程设计研究院有限公司',
  'Location': '上海-虹口区',
  'Application Time': '2024-04-08',
  'Resume Status': '已到达'},
 {'Jo

In [24]:
# 遍历每个元素，提取并打印/保存你感兴趣的信息
for element in elements:
    job_title = element.find_element(By.CSS_SELECTOR, "a.zhn.gt").text
    salary = element.find_element(By.CSS_SELECTOR, "span.xz").text
    company = element.find_element(By.CSS_SELECTOR, "a.gs").text
    location = element.find_element(By.CSS_SELECTOR, "span.dq").text
    print(f"Job Title: {job_title}, Salary: {salary}, Company: {company}, Location: {location}")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a.zhn.gt"}
  (Session info: chrome=123.0.6312.122); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF788BFAD32+56930]
	(No symbol) [0x00007FF788B6F632]
	(No symbol) [0x00007FF788A242E5]
	(No symbol) [0x00007FF788A698ED]
	(No symbol) [0x00007FF788A69A2C]
	(No symbol) [0x00007FF788A5F13C]
	(No symbol) [0x00007FF788A8BCDF]
	(No symbol) [0x00007FF788A5F09A]
	(No symbol) [0x00007FF788A8BEB0]
	(No symbol) [0x00007FF788AA81E2]
	(No symbol) [0x00007FF788A8BA43]
	(No symbol) [0x00007FF788A5D438]
	(No symbol) [0x00007FF788A5E4D1]
	GetHandleVerifier [0x00007FF788F76ABD+3709933]
	GetHandleVerifier [0x00007FF788FCFFFD+4075821]
	GetHandleVerifier [0x00007FF788FC818F+4043455]
	GetHandleVerifier [0x00007FF788C99766+706710]
	(No symbol) [0x00007FF788B7B90F]

In [18]:
elements = driver.find_elements(By.CSS_SELECTOR, ".e")

In [19]:
for element in elements:
    print(element.text)









光伏技术工程师(J10083)
7-8.5千·18薪
立即沟通

深圳创维光伏科技有限公司
深圳-光明区
申请于2024-04-08
申请简历：我的简历1
近两月投递中
你超过了??%竞争者
我的竞争力分析
已到达
数据分析专员
8-9千
立即沟通

上海共添市场营销服务股份有限公司
上海
申请于2024-04-08
申请简历：我的简历1
近两月投递中
你超过了??%竞争者
我的竞争力分析
已到达
项目经理
8-9千
立即沟通

深圳高速公路石油有限公司
深圳-福田区
申请于2024-04-08
申请简历：我的简历1
近两月投递中
你超过了??%竞争者
我的竞争力分析
已到达
项目助理
7-8千·15薪
立即沟通

森欧汽车内饰材料（江苏）有限公司
广州-番禺区
申请于2024-04-08
申请简历：我的简历1
近两月投递中
你超过了??%竞争者
我的竞争力分析
已查阅
c证货运司机
1.3-1.8万
立即沟通

上海瑜源货物运输代理有限公司
上海
申请于2024-04-08
申请简历：我的简历1
近两月投递中
你超过了??%竞争者
我的竞争力分析
已到达
IT基础架构工程师
1-1.5万
立即沟通

上海电子工程设计研究院有限公司
上海-虹口区
申请于2024-04-08
申请简历：我的简历1
近两月投递中
你超过了??%竞争者
我的竞争力分析
已到达
软件工程师
1-2万
立即沟通

深圳市西迪特科技股份有限公司
深圳-南山区
申请于2024-04-08
申请简历：我的简历1
近两月投递中
你超过了??%竞争者
我的竞争力分析
已到达
Java开发工程师（招应届生）
6千-1.2万·15薪
立即沟通

天信达信息技术有限公司
上海-闵行区
申请于2024-04-08
申请简历：我的简历1
近两月投递中
你超过了??%竞争者
我的竞争力分析
已到达
SFE Manager
2-3万
立即沟通

豪雅捷美士（宁波）医疗器械有限公司
上海-徐汇区
申请于2024-04-08
申请简历：我的简历1
近两月投递中
你超过了??%竞争者
我的竞争力分析
已到达
抗体研发助理（生物医药大分子创新平台）
6千-1.2万
立即沟通

中国医药工业研究总院有限公司
上海-浦东新区
申请于2024-04-08
申请简历：我的简历1
近两月投递中
你超过了??%竞争者


In [21]:
for element in elements:
    try:
        job_title = element.find_element(By.CSS_SELECTOR, "a.zhn.gt").text
        salary = element.find_element(By.CSS_SELECTOR, "span.xz").text
        company = element.find_element(By.CSS_SELECTOR, "a.gs").text
        location = element.find_element(By.CSS_SELECTOR, "span.dq").text
        print(f"Job Title: {job_title}, Salary: {salary}, Company: {company}, Location: {location}")
    except:
        continue



Job Title: 光伏技术工程师(J10083), Salary: 7-8.5千·18薪, Company: 深圳创维光伏科技有限公司, Location: 深圳-光明区
Job Title: 数据分析专员, Salary: 8-9千, Company: 上海共添市场营销服务股份有限公司, Location: 上海
Job Title: 项目经理, Salary: 8-9千, Company: 深圳高速公路石油有限公司, Location: 深圳-福田区
Job Title: 项目助理, Salary: 7-8千·15薪, Company: 森欧汽车内饰材料（江苏）有限公司, Location: 广州-番禺区
Job Title: c证货运司机, Salary: 1.3-1.8万, Company: 上海瑜源货物运输代理有限公司, Location: 上海
Job Title: IT基础架构工程师, Salary: 1-1.5万, Company: 上海电子工程设计研究院有限公司, Location: 上海-虹口区
Job Title: 软件工程师, Salary: 1-2万, Company: 深圳市西迪特科技股份有限公司, Location: 深圳-南山区
Job Title: Java开发工程师（招应届生）, Salary: 6千-1.2万·15薪, Company: 天信达信息技术有限公司, Location: 上海-闵行区
Job Title: SFE Manager, Salary: 2-3万, Company: 豪雅捷美士（宁波）医疗器械有限公司, Location: 上海-徐汇区
Job Title: 抗体研发助理（生物医药大分子创新平台）, Salary: 6千-1.2万, Company: 中国医药工业研究总院有限公司, Location: 上海-浦东新区
Job Title: 高级需求分析师, Salary: 1.5-2万, Company: 广州宝立科技有限公司, Location: 广州-黄埔区
Job Title: AI训练师, Salary: 1-2万·13薪, Company: 深圳市融臻科技管理有限公司, Location: 深圳-福田区
Job Title: 技术服务工程师, Salary: 8千-1万, Company: 上

In [13]:
job_title

NameError: name 'job_title' is not defined

In [5]:
#翻页功能
def page_turning(driver):

    """
    遍历页面的分页。
    
    :param driver: WebDriver实例，用于页面操作。
    :param combination: 用于标识不同文件夹/文件组合的标识符。

    """

    #folder_path = os.path.join(os.getcwd(), str(combination))  # 构建文件夹路径

    try:
        last_page_element = driver.find_element(By.CSS_SELECTOR, "ul.el-pager li.number:last-child")
        total_pages = int(last_page_element.text)
    except NoSuchElementException:
        print("无法找到分页元素，检查选择器或页面结构。")
        return 

    for page in range(1, total_pages + 1):
        if page > 1:
            try:
                jump_page = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.ID, "jump_page"))
                )
                jump_page.clear()
                jump_page.send_keys(page)
                
                jump_button = driver.find_element(By.CSS_SELECTOR, ".jumpPage")
                jump_button.click()
                sleep_time=random.uniform(0.5,2.0)
                sleep(sleep_time)

                # if check_page_loaded(driver, timeout=60, poll_frequency=0.5):
                #     if not wait_for_file(folder_path, page):
                #         print(f"等待页面 {page} 对应的文件超时，文件未在文件夹 {folder_path} 中找到。")
                #         driver.refresh()  # 刷新一下，重新抓取
                

            except (NoSuchElementException, TimeoutException) as e:
                print(f"在尝试跳转到页面 {page} 时发生错误: {e}")
                break  # 如果出现错误，终止循环   
    return 


In [6]:
page_turning(driver)

无法找到分页元素，检查选择器或页面结构。


In [7]:
test_url=f"https://i.51job.com/userset/compete.php?jobid=154254982&resumeid=468287804&accountid=241451461&lang=c"

driver.get(test_url)





In [9]:
element_text = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[2]/div[1]/strong[2]').text

print(element_text)


（排名135/386）


In [4]:
data=read_json_file(r"C:\Users\Xue\Documents\GitHub\gender-discrimination-hiring\added.json")


In [29]:


for job in data:
    if "competition" in job:
        continue
    else:
        competition_url=f"https://i.51job.com/userset/compete.php?jobid={job["jobId"]}&resumeid=468287804&accountid=241451461&lang=c"

        driver.get(competition_url)

        #加一个等待加载的
        sleep_time=random.uniform(1.5,2.5)
        sleep(sleep_time)
        try:
            element_text = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[2]/div[1]/strong[2]').text
            competition=get_competition(element_text)
            job['competition']=competition
        except:
            print(job['jobHref'])



https://jobs.51job.com/beijing/147183192.html?s=sou_sou_soulb&t=0_0&req=97306dde94d40c631dd5c171d0d8d672
https://jobs.51job.com/shanghai/150769406.html?s=sou_sou_soulb&t=0_0&req=32f831e9dc5e84f3481ffc4ab4f247a5


In [5]:
merge=r"C:\Users\Xue\OneDrive\0_Code\data\2024-4-6-1800-2000\merge.json"

final_data=r"C:\Users\Xue\OneDrive\0_Code\data\final_processed_jobs_or.json"

In [6]:
data2=read_json_file(merge)


In [7]:

data3=read_json_file(final_data)

In [9]:
jobIds=[]
for job in data3:
    jobIds.append(job["jobId"])
jobHerfs=set()
InSchool_jobIds=[]


i=0
for job in data2:
    if job["jobId"] in jobIds:
        if "在校生/应届生" in job["workYearString"]:
            InSchool_jobIds.append(job["jobId"])

for job in data3:
    if job["jobId"] in InSchool_jobIds:
        job["workYearString"]="在校生/应届生"
    else :
        job["workYearString"]=""

    monthly_salary = calculate_monthly_salary_true(job["provideSalaryString"])
    if monthly_salary is not None:
        job["monthly_salary"] = monthly_salary
    else:
        job["monthly_salary"] = -1


# 将处理后的数据保存到新的 JSON 文件中
with open('final_processed_jobs_workyear.json', 'w', encoding='utf-8') as new_file:
    json.dump(data3, new_file, ensure_ascii=False, indent=4)



            # print(job["jobHref"])
            
            # jobHerfs.add(job["jobHref"])
            # i=i+1


# print(i)
# j=0
# for job in jobHerfs:
#     j=j+1
# print(j)


In [19]:
for job in data:
   if competition not in job:
      print(job)

{'jobId': '151227315', 'jobName': '系统与软件建模工程师', 'jobAreaString': '北京', 'provideSalaryString': '7-9千·14薪', 'issueDateString': '2024-04-04 16:44:16', 'workYearString': '1年', 'degreeString': '本科', 'fullCompanyName': '北京火龙果网络技术有限公司', 'companyTypeString': '民营', 'companySizeString': '少于50人', 'jobHref': 'https://jobs.51job.com/beijing-hdq/151227315.html?s=sou_sou_soulb&t=0_0&req=3738bc51edb5e11634a60dda48212b68', 'termStr': '全职', 'jobWelfareCodeDataList': ['节日福利', '交通补贴', '餐饮补贴', '生日福利', '公司旅游', '专业培训', '通讯补贴', '五险一金'], 'monthly_salary': 46083.33, 'monthly_salary_true': 9333.33, 'competition': '11'}
{'jobId': '153707996', 'jobName': '工程师/助理工程师', 'jobAreaString': '上海', 'provideSalaryString': '6.5-9千·15薪', 'issueDateString': '2024-03-23 04:00:41', 'workYearString': '在校生/应届生', 'degreeString': '本科', 'fullCompanyName': '上海文广科技（集团）有限公司', 'companyTypeString': '国企', 'companySizeString': '150-500人', 'jobHref': 'https://jobs.51job.com/shanghai-xhq/153707996.html?s=sou_sou_soulb&t=0_0&req=4021c4bbe1aba9

In [ ]:
test_url=f"https://i.51job.com/userset/compete.php?jobid=154254982&resumeid=468287804&accountid=241451461&lang=c"

driver.get(test_url)

#拿取jobId的



#加一个等待加载的
sleep_time=random.uniform(0.5,2.0)
sleep(sleep_time)

#抓取，保存

element_text = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[2]/div[1]/strong[2]').text

print(element_text)

In [ ]:


# 设置日志记录的配置
logging.basicConfig(filename='script.log', filemode='a', format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)




""" # 替换为你的job IDs文件路径
jobids_file_path = 'jobids.txt'


# 读取job IDs
with open(jobids_file_path, 'r') as file:
    jobids = [line.strip() for line in file if line.strip()]
 """




data=read_json_file(r"C:\Users\Xue\Documents\GitHub\gender-discrimination-hiring\added.json")

jobids=[]
for job in data:
    jobids.append(job["jobId"])




for job in data:
    base_url=f"https://i.51job.com/userset/compete.php?jobid={job["jobId"]}&resumeid=468287804&accountid=241451461&lang=c"




# 设置日志记录的配置
logging.basicConfig(filename='script.log', filemode='a', format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)





from selenium.common.exceptions import NoSuchElementException

def fetch_job_details(driver, date, job_type):
    # 等待页面加载
    wait = WebDriverWait(driver, 10)
    
    job_url = driver.current_url

    try:
        job_title = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div/div[1]/h1").get_attribute('title')

    except NoSuchElementException:
        input_flag = int(input("遇到意料外的页面需要处理(1:已处理/2:退出当前页面处理下一条):"))
        if input_flag == 0:
            print("未排除错误, 已关闭并存储, 读取下一条")
            write_error_urls(job_url)
            return {}
        else:
            try:
                job_title = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div/div[1]/h1").get_attribute('title')
            except NoSuchElementException:
                print("未排除错误, 已关闭并存储, 读取下一条")
                write_error_urls(job_url)
                return {}

    
    company_name = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[4]/div[2]/div[1]/a/p").get_attribute('title')
    
    salary = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div/div[1]/strong").text
    
    details = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div/div[1]/p').get_attribute('title')
    # 由于原始字符串使用了"&nbsp;"作为空格和"|&nbsp;"作为分隔符，可能需要替换或直接分割
    print(details)
    details_parts = details.replace('\xa0\xa0', ' ').split(' | ')
    print(details_parts)
    work_area = None
    experience = None
    education = None

    if len(details_parts) > 0:
        work_area = details_parts[0]
    if len(details_parts) > 1:
        experience = details_parts[1]
    if len(details_parts) > 2:
        education = details_parts[2]

    company_type = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[4]/div[2]/div[2]/p[1]").get_attribute('title')
    
    is_intern = ""
    
    issue_date = date
    
    # 使用find_elements来找到所有匹配的<span>元素
    benefits_elements = driver.find_elements(By.XPATH, "/html/body/div[2]/div/div[2]/div/div[1]/div/div/span")

    # 遍历元素集合，提取每个元素的文本
    job_welf = [element.text for element in benefits_elements]
    
    requirement = [experience, education,""]
    
    company_size = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[4]/div[2]/div[2]/p[2]").get_attribute('title')
    
    company_industry = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[4]/div[2]/div[2]/p[3]").get_attribute('title')

    # 收集到的数据存入字典
    job_data = {
        "job_title": job_title,
        "company_name": company_name,
        "salary": salary,
        "work_area": work_area,
        "company_type": company_type,
        "is_intern": is_intern,
        "issue_date": issue_date,
        "job_welf": job_welf,
        "requirement": requirement,
        "company_size": company_size,
        "company_industry": company_industry,
        "job_url": job_url,
        "job_type":job_type
        # 添加更多字段...
    }
    print(job_data)
    
    return job_data

def wait_for_element_clickable(driver, by_method, selector, timeout=10):

    return WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((by_method, selector))
    )

def crawl_current_page(driver, jobs_data, times):
    # 定位到职位元素列表
    job_elements = driver.find_elements(By.CSS_SELECTOR, ".joblist-item")

    for job_element in job_elements:

        # 从元素获取sensorsdata属性，它是一个JSON格式的字符串
        job_elements_detail = job_element.find_element(By.CSS_SELECTOR, ".joblist-item-job")
        sensorsdata = job_elements_detail.get_attribute("sensorsdata")
        
        # 将JSON字符串解析为Python字典
        data = json.loads(sensorsdata.replace('&quot;', '"'))
        
        # 提取jobTime属性
        issue_time = data.get("jobTime")
        job_type = data.get("funcType")
        job_title = data.get("jobTitle")
        salary = data.get("jobSalary")
        work_area = data.get("jobArea")
        experience = data.get("jobYear")
        education = data.get("jobDegree")
        is_intern = ""
        
        try:
            job_tag = job_element.find_element(By.CSS_SELECTOR, ".tags")
            job_tags = job_tag.find_elements(By.CSS_SELECTOR, ".tag")
            job_welf = [element.text for element in job_tags]
        except:
            job_welf = []

        # 遍历元素集合，提取每个元素的文本
        
        requirement = [experience, education,""]

        company_elements_detail = job_element.find_element(By.CSS_SELECTOR, ".joblist-item-bot")

        company_name = company_elements_detail.find_element(By.CSS_SELECTOR, ".cname").text
        try:
            company_industry = company_elements_detail.find_element(By.CSS_SELECTOR, 'span.dc.text-cut').text
            try:
                company_type = company_elements_detail.find_elements(By.CSS_SELECTOR, 'span.dc.shrink-0')[0].text
            except:
                company_type = ''
            # 找到公司规模元素
            try:
                company_size = company_elements_detail.find_elements(By.CSS_SELECTOR, 'span.dc.shrink-0')[1].text
            except:
                company_size = ''
        except:
            try:
                company_type = company_elements_detail.find_elements(By.CSS_SELECTOR, 'span.dc.shrink-0')[1].text
            except:
                company_type = ''
            # 找到公司规模元素
            try:
                company_size = company_elements_detail.find_elements(By.CSS_SELECTOR, 'span.dc.shrink-0')[2].text
            except:
                company_size = ''
            try:
                company_industry = company_elements_detail.find_element(By.CSS_SELECTOR, 'span.dc.shrink-0')[0].text
            except:
                company_industry = ''
        # 找到公司类型元素
        

        # 获取当前所有打开的窗口句柄
        original_window_handles = driver.window_handles
        original_window = driver.current_window_handle

        # 模拟点击每个职位元素
        try:
            job_element.find_element(By.CSS_SELECTOR, ".joblist-item-top").click()
            print("click")
        except:
            print("click-error")
        sleep(1)
        wait = WebDriverWait(driver, 10)
        
        # 检查是否有新窗口打开
        new_window_handles = driver.window_handles
        new_windows = set(new_window_handles) - set(original_window_handles)

        if new_windows:
            print()        
        else:
             # 没有新窗口打开，等待10秒
            print("No new window opened. Waiting for 10 seconds.")
            sleep(10)

        current_url = ""
        try:             #change
            driver.switch_to.window(driver.window_handles[1])
            current_url = driver.current_url
        except:
            current_url = ""

        data = {
            "job_title": job_title,
            "company_name": company_name,
            "salary": salary,
            "work_area": work_area,
            "company_type": company_type,
            "is_intern": is_intern,
            "issue_date": issue_time,
            "job_welf": job_welf,
            "requirement": requirement,
            "company_size": company_size,
            "company_industry": company_industry,
            "job_url": current_url,
            "job_type":job_type
            # 添加更多字段...
        }

        jobs_data.append(data)

        print("成功抓取一个岗位信息")  # 输出调试信息
        print(data)
        if current_url != "":
            driver.close()  # 关闭当前的职位详细信息窗口
        driver.switch_to.window(driver.window_handles[0])  # 切换回职位列表的窗口

                    

        times = times - 1
        print("剩余:"+str(times))
        if times == 0:
            return times

    return times


# 基础URL
base_url = "https://we.51job.com/pc/search?searchType=2&sortType=0&metro="

completed_combinations_set = set()

try:
    with open(r'C:\Users\Xue\OneDrive\0_Code\demo\completed_combinations.txt', 'r') as file:
        for line in file:
            completed_combinations_set.add(line.strip())
    print("completed combinations file found")
except FileNotFoundError:
    print("No completed combinations file found, starting fresh.")


for list_name, function_codes in function_IT.items():
    for function in function_codes:
        for area in jobArea:
            if (function in ('0107', '0106', '0121', '0156', '0157', '0153', '0126', '0152', '0124', '0120')):
                continue

            combination = f"{area}-{function}"
            
            if combination in completed_combinations_set:
                continue  # 跳过已完成的组合

            # 构建完整的URL
            url = f"{base_url}&jobArea={area}&function={function}" 
            #print("Visiting URL:", url)
            # 发送GET请求
            #urls.append(url)

            # 初始化Chrome WebDriver
            driver = webdriver.Chrome(options=options)

            # 打开目标网页
            driver.get(url)
            sleep(1)
            driver.refresh()
            sleep(5)

            

            try:
                # 尝试找到表示没有职位的特定文本元素
                no_jobs_element = driver.find_element(By.XPATH, "//*[contains(text(), '哦哦！没有职位不要怕，你那么年轻那么好看，再重搜一次呗~')]")
                print("没有找到职位，跳过此次循环。")
                with open('completed_combinations.txt', 'a') as file:
                    file.write(combination + '\n')
                completed_combinations_set.add(combination)
                print(f"组合已记录至completed_combinations.txt文件。")
                continue  # 如果找到了该元素，说明没有职位，跳过本次循环
            except NoSuchElementException:
                # 如果没有找到特定文本，说明页面上有职位列表，继续执行后面的代码
                print("职位列表存在，继续处理。")            
            
            # 确保首页职位列表被加载
            wait_for_element_clickable(driver, By.CSS_SELECTOR, ".joblist-item-job")




            #这里可以读取
            # 假设您已经有了一个名为driver的WebDriver实例
            last_page_element = driver.find_element(By.CSS_SELECTOR, "ul.el-pager li.number:last-child")
            total_pages = int(last_page_element.text)



            for page in range(1, total_pages + 1):   
                if page > 1:  # 如果不是第一页，找到分页输入框，输入页码，然后点击跳转
                    jump_page = wait_for_element_clickable(driver, By.ID, "jump_page")
                    jump_page.clear()
                    jump_page.send_keys(page)
                    
                    jump_button = driver.find_element(By.CSS_SELECTOR, ".jumpPage")
                    jump_button.click()
                    
                    # 等待页面加载完毕
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, ".joblist-item-job"))
                    )
                
                times = crawl_current_page(driver, jobs_data, times)
                if times == 0:
                    break 

                print(f"完成第 {page} 页的抓取")
                

            # 完成所有任务后关闭浏览器
            driver.quit()

            # 使用新的文件名写入数据
            with open(filename, 'w', encoding='utf-8') as f:
                # 确保使用`ensure_ascii=False`以支持中文字符，pretty print 用于美化输出
                json.dump(jobs_data, f, ensure_ascii=False, indent=4)

            print(f"数据已导出到 '{filename}' 文件。")
            with open('completed_combinations.txt', 'a') as file:
                file.write(combination + '\n')

            completed_combinations_set.add(combination)
            print(f"组合已记录至completed_combinations.txt文件。")

#            except:



    jobs_data.clear()

